# Snap - Enron email network

Test with a [snap dataset](https://snap.stanford.edu/data/).

Tested with the [Enron email network](https://snap.stanford.edu/data/email-Enron.html)

Characteristics of this graph:
- Nodes: 36692
- Edges: 183831
- Nodes in largest WCC: 33696 (0.918)

In [1]:
import pandas as pd

In [2]:
bm1=pd.read_csv('https://snap.stanford.edu/data/email-Enron.txt.gz',
            skiprows=4,
            header=None,
            names=['a','b'],
            #nrows=1000,
            sep='\t')
bm1.head()

,a,b
0,0,1
1,1,0
2,1,2
3,1,3
4,1,4


In [3]:
bm1.shape

(367662, 2)

In [4]:
from connectedComponentsLabeling import connectedComponentsLabeler as ccl
from time import time
import pandas as pd
import numpy as np
import duckdb

In [5]:
%%time 

startTime=time()

yoba_cc=ccl(bm1).getConnectedCompontents()

yobaTime=time()-startTime

CPU times: user 1.2 s, sys: 12.7 ms, total: 1.21 s
Wall time: 1.21 s


In [6]:
yoba_cc.head()

,nodeId,componentId
0,0,36624
1,1,36624
2,2,36624
3,3,36624
4,4,36624


In [7]:
%%time 

duckdb.sql('''
select count(distinct componentId)
from yoba_cc
''')

CPU times: user 1.25 ms, sys: 952 μs, total: 2.21 ms
Wall time: 1.88 ms


┌─────────────────────────────┐
│ count(DISTINCT componentId) │
│            int64            │
├─────────────────────────────┤
│                        1065 │
└─────────────────────────────┘

In [9]:
%%time 

duckdb.sql('''
select componentId, count(nodeId) as ms_countOf_nodes
from yoba_cc
group by 1
order by 2 desc
limit 5
''')

CPU times: user 2.93 ms, sys: 0 ns, total: 2.93 ms
Wall time: 1.96 ms


┌─────────────┬──────────────────┐
│ componentId │ ms_countOf_nodes │
│   uint32    │      int64       │
├─────────────┼──────────────────┤
│       36624 │            33696 │
│       30290 │               20 │
│       34603 │               16 │
│       36147 │               14 │
│       29769 │               13 │
└─────────────┴──────────────────┘